In [1]:
%load_ext autoreload
%matplotlib ipympl
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
import bpy
from bpy.types import bpy_prop_array
import mathutils
import numpy as np
import clipboard
import json

In [2]:
import sys
sys.path.append('/Users/orenm/BlenderShaderProject/project_files/')

In [3]:
%autoreload 2
from Logic.node_readers_writers import *
from Logic.network_manager import NetworkManager
from Logic.meta_network import *
from Logic.structures_definitions import *
from Logic.variations_creator import *

In [13]:
self = NetworkManager()
self.initialize_network()
self.generate_random_network(n_additions=4)
self.finish_network()

code = self.generate_code(with_initialization_code=True)
clipboard.copy(code)
# self.draw_network()

In [23]:
def save_graph(graph, filename):
    with open(filename, 'w') as f:
        json.dump(nx.node_link_data(graph, edges='edges'), f)

def load_graph(filename):
    with open(filename, 'r') as f:
        return nx.node_link_graph(json.load(f), edges='edges')

In [8]:
exec(code)

Info: Deleted 3 data-block(s)


In [ ]:
def save_graph_with_backup(graph, filename):
    if os.path.exists(filename):
        backup_filename = f"{filename}.backup"
        os.rename(filename, backup_filename)
        print(f"Existing file backed up as: {backup_filename}")

    with open(filename, 'w') as f:
        json.dump(nx.node_link_data(graph), f)

def append_to_graph_log(graph_update, log_filename):
    with open(log_filename, 'a') as f:
        json.dump(graph_update, f)
        f.write('\n')  # Separate entries by newline

def reconstruct_graph_from_log(log_filename):
    G = nx.Graph()
    with open(log_filename, 'r') as f:
        for line in f:
            update = json.loads(line)
            G.add_nodes_from(update.get('nodes', []))
            G.add_edges_from(update.get('edges', []))
    return G

In [41]:
manager = MetaNetworkManager(ALL_META_NODES, max_layers=2, n_additions=3)
manager.generate_network()
nm = manager.meta_network_to_flat_network()

nodes_to_remove = [x for x in nm.network.nodes() if x not in ['OutputNode_1', 'InputNode_1']]
np.random.shuffle(nodes_to_remove)

nm_for_change = nm.copy()
# get a random order of all nodes that need to be removed until we have an empty network
process = []
for node in nodes_to_remove:
    twoway_variation = create_remove_node_variation(nm_for_change, node)
    process.append(twoway_variation)
    for variation in twoway_variation.steps_forward:
        apply_variation(nm_for_change, variation)

for sub_process in reversed(process):
    for variation in sub_process.steps_backward:
        apply_variation(nm_for_change, variation)

In [ ]:
def change_seed(nm):
    return non_structural_changes(nm, 3, ParamRequestType.SEED)

def change_numeric(nm):
    return non_structural_changes(nm, 3, ParamRequestType.NUMERIC)

def change_params(nm):
    return non_structural_changes(nm, 3, ParamRequestType.NON_SEED)

funcs = [change_seed, change_numeric, change_params, add_random_node_on_edge, add_random_edge, remove_random_edge, remove_random_node]


In [8]:
for i in range(1000):
    manager = MetaNetworkManager(ALL_META_NODES, max_layers=2, n_additions=3)
    manager.generate_network()
    nm = manager.meta_network_to_flat_network()

    for i in range(5):
        func = np.random.choice(funcs)
        variation_desc = func(nm)
        for variation in variation_desc.steps_forward:
            apply_variation(nm, variation)
    
    old_network = nm.copy()
    func = np.random.choice(funcs)   
    variation_desc = func(nm)
    for variation in variation_desc.steps_forward:
        apply_variation(nm, variation)
    
    for variation in variation_desc.steps_backward:
        apply_variation(nm, variation)
    
    should_compare_props = func in [change_seed, change_numeric, change_params]
    nodes, edges = nm.network_data_for_comparison()
    old_nodes, old_edges = old_network.network_data_for_comparison()
    assert NetworkManager.compare_networks(nodes, edges, old_nodes, old_edges, compare_node_properties=True)